In [ ]:
!pip install -q huggingface_hub
!pip install -q -U trl peft
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 bitsandbytes==0.41.1
!pip install -q -U datasets wandb


In [ ]:
# Uncomment to install new features that support latest models like Llama 2
# !pip install git+https://github.com/huggingface/peft.git
# !pip install git+https://github.com/huggingface/transformers.git

# When prompted, paste the HF access token you created earlier.
from huggingface_hub import notebook_login
notebook_login()



In [ ]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

dataset_name = "Kirili4ik/yandex_jobs"
dataset = load_dataset(dataset_name, split='train')


/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [ ]:
dataset[0]

{'Header': 'Старший Java-разработчик в Музыку',
 'Emoji': '🎧',
 'Description': 'Вас ждет работа с высоконагруженными системами обработки больших объемов данных и крупнейшей в России базой легальной музыки. Будет много хорошей музыки и качественного кода.',
 'Requirements': '• уверенное знание Java, СУБД, Linux\n• умение тестировать собственный код и разбираться в чужом',
 'Tasks': '• проектировать новые фичи, поддерживать и развивать уже существующие',
 'Pluses': '• опыт разработки распределенных систем с большой нагрузкой\n• опыт использования реляционных СУБД или NoSQL-хранилищ (MongoDB)\n• опыт многопоточного программирования',
 'Hashtags': '#фронтенд #java #senior',
 'Link': 'https://ya.cc/t/26NhxwD4CH6Ur',
 'Raw text': 'Старший Java-разработчик в Музыку🎧\n\nВас ждет работа с высоконагруженными системами обработки больших объемов данных и крупнейшей в России базой легальной музыки. Будет много хорошей музыки и качественного кода.\n\nМы ждем, что вы:\n• уверенное знание Java, СУБД, 

In [ ]:
def preprocess(row):
  row['header_req'] = f"Требования на позицию {row['Header']}: \n {row['Requirements']}"
  return row

In [ ]:
prep_dataset = dataset.map(preprocess)

In [ ]:
base_model_name = "meta-llama/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    device_map='auto',
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True

)

device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:479: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:
torch.cuda.empty_cache()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

output_dir = "model"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    max_steps=125,
)

max_seq_length = 100

trainer = SFTTrainer(
    model=base_model,
    train_dataset=prep_dataset,
    peft_config=peft_config,
    dataset_text_field="header_req",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
)


Map:   0%|          | 0/625 [00:00<?, ? examples/s]

In [ ]:
base_model = None

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = tokenizer.encode("Python разработчик", return_tensors="pt").to(device)
outputs = base_model.generate(inputs)

In [ ]:
base_model.push_to_hub('')

129

In [ ]:
trainer.train()

import os
output_dir = os.path.join(output_dir, "final_checkpoint")
trainer.model.save_pretrained(output_dir)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.244000
20,1.744400
30,1.485500
40,1.303000
50,1.226100
60,1.191200
70,1.120200
80,1.113300
90,1.082000
100,1.062300


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
inputs.device, base_model.device

(device(type='cuda', index=0), device(type='cuda', index=0))

In [ ]:
tokenizer

LlamaTokenizerFast(name_or_path='meta-llama/Llama-2-7b-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
inputs = tokenizer(["Требования к позиции Python разработчик: ",
                    "Требования к позиции data scientist:",
                    "Требования к позиции project manager:"], padding=True, return_tensors="pt")


In [ ]:
# base_model.to(device)
# inputs = tokenizer(["Требования к позиции Python разработчик: ", ], return_tensors="pt").to(device)
with torch.no_grad():
  inputs = {k: v.to(device) for k, v in inputs.items()}
  outputs = base_model.generate(**inputs, max_new_tokens=100)
  for el in tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True):
    print(el)

Требования к позиции Python разработчик: 
 • больше двух лет занимались разработкой на Python
• работали с реляционными СУБД и NoSQL-базами данных
• работали с большими объемами данных
• разрабатывали решения с использованием фреймворков: Django, Flask, Pyramid, Tornado или с использованием фреймворков на C++
• работали с СУБД
Требования к позиции data scientist: • уверенно владеете Python
պ. 300-350
• знаете математику, алгоритмы, теорию вероятностей
• знаете SQL
• знаете методы анализа данных
• работали с большими данными
• работали с ML-технологиями
• знаете ML-технологии, которые мы используем на работе

Требования к позиции project manager: • знаете Java, C++, Python, Go или другой современный язык программирования
Љакаете проектировать и разрабатывать сложные системы
• знаете классические алгоритмы и структуры данных
• умеете писать читаемый код
• любите работать в команде, умеете понимать и удовлетворять потре


In [ ]:
with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = base_model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

OutOfMemoryError: ignored

In [ ]:
base_model.push_to_hub('ai-ar/llama2_finetune')

NotImplementedError: ignored